In [7]:
import os
import shutil
from docx import Document
from PIL import Image
import pytesseract
import PyPDF2
import os

In [8]:
file_path = 'my_img2.jpg'

In [9]:
if os.access(file_path, os.R_OK):
    print("File is readable")
else:
    print("File is not readable")

File is readable


In [10]:
pytesseract.pytesseract.tesseract_cmd = r"E:\Tesseract\tesseract.exe"

In [17]:
class FileClassifier:
    def __init__(self, keywords_by_category):
        self.keywords_by_category = keywords_by_category

    def classify_file(self, file_path):
        _, file_extension = os.path.splitext(file_path)
        file_extension = file_extension.lower()[1:]  # Remove the leading dot and convert to lowercase

        if file_extension == 'pdf':
            return self.classify_pdf(file_path)
        elif file_extension in ('doc', 'docx'):
            return self.classify_doc(file_path)
        elif file_extension in ('png', 'jpg'):
            return self.classify_image(file_path)
        else:
            print(f"Unsupported file format: {file_extension}")
            return None

    def classify_pdf(self, pdf_path):
        # Implementation for PDF classification (same as before)
        try:
            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ''
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()

                return self.classify_text(text)

        except Exception as e:
            print(f"Error processing PDF '{pdf_path}': {e}")
            return None

    def classify_doc(self, doc_path):
        # Implementation for DOC/DOCX classification (same as before)
        try:
            doc = Document(doc_path)
            text = ''
            for para in doc.paragraphs:
                text += para.text

            return self.classify_text(text)

        except Exception as e:
            print(f"Error processing DOC/DOCX '{doc_path}': {e}")
            return None

    def classify_image(self, image_path):
        try:
            # Use pytesseract to perform OCR on the image and extract text
            text = pytesseract.image_to_string(Image.open(image_path))

            return self.classify_text(text)

        except Exception as e:
            print(f"Error processing image '{image_path}': {e}")
            return None

    def classify_text(self, text):
        for category, keywords in self.keywords_by_category.items():
            for keyword in keywords:
                if keyword.lower() in text.lower():
                    return category
        return "Unclassified"
    
if __name__ == "__main__":
    keywords_by_category = {
        'Cricket': ['Cricket', 'keyword2', 'keyword3'],
        'Football': ['Football', 'keyword5'],
        'Handball': ['Handball', 'keyword7', 'keyword8']
    }

    file_classifier = FileClassifier(keywords_by_category)

    folder_path = 'Testing/'
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            result = file_classifier.classify_file(file_path)
            if result:
                print(f"The file '{file_path}' belongs to the category: {result}")

                # Create folder if not exists
                category_folder = os.path.join(folder_path, result)
                if not os.path.exists(category_folder):
                    os.makedirs(category_folder)

                # Move file to category folder
                shutil.move(file_path, os.path.join(category_folder, filename))
            else:
                print(f"Unable to classify the file '{file_path}'.")


The file 'Testing/my_img2.jpg' belongs to the category: Handball
The file 'Testing/my_img3.jpg' belongs to the category: Cricket
The file 'Testing/my_img_1.jpg' belongs to the category: Handball
The file 'Testing/my_pdf2.pdf' belongs to the category: Cricket
The file 'Testing/my_pdf3.pdf' belongs to the category: Handball
The file 'Testing/my_pdf4.pdf' belongs to the category: Handball
The file 'Testing/my_word2.docx' belongs to the category: Cricket
The file 'Testing/my_word3.docx' belongs to the category: Cricket
The file 'Testing/my_word4.docx' belongs to the category: Football
